In [1]:
!pip install --upgrade pip

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\andre\miniconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))   

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, lh, rh])

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'nothing', 'help', 'yes', 'bathroom'])
# actions = np.array(['hello'])
# actions = np.array(['thanks'])
# actions = np.array(['nothing'])
# actions = np.array(['help'])
# actions = np.array(['yes'])
# actions = np.array(['bathroom'])    # Use Left hand


# Number of new sequences to record per action
new_videos = 50

# Videos are going to be 30 frames in length
sequence_length = 30  

In [9]:
def get_start_sequence(action):
    action_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_path):
        return 0  # If action folder doesn't exist, start from 0
    existing_sequences = [int(seq) for seq in os.listdir(action_path) if seq.isdigit()]
    return max(existing_sequences) + 1 if existing_sequences else 0

In [ ]:
cap = cv2.VideoCapture(0)  # Open webcam

with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic:

    for action in actions:
        start_sequence = get_start_sequence(action)  # Get last recorded sequence number
        print(f"Starting collection for '{action}' from sequence {start_sequence}")

        for sequence in range(start_sequence, start_sequence + new_videos):
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)), exist_ok=True)

            for frame_num in range(sequence_length):
                ret, frame = cap.read()  # Capture frame
                
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Show collection status
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action}, Video {sequence}', (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)  # Delay before recording
                else:
                    cv2.putText(image, f'Collecting frames for {action}, Video {sequence}', (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Show feed
                cv2.imshow('OpenCV Feed', image)

                # Press 'q' to quit
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()

# Release resources
cap.release()
cv2.destroyAllWindows()


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential([
    Input(shape=(30, 258)),
    LSTM(64, return_sequences=True, activation='tanh'),
    LSTM(128, return_sequences=True, activation='tanh'),
    LSTM(64, return_sequences=False, activation='tanh'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(actions.shape[0], activation='softmax')
])

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - categorical_accuracy: 0.4143 - loss: 1.3957
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - categorical_accuracy: 0.7258 - loss: 0.6511
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - categorical_accuracy: 0.8475 - loss: 0.4391
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - categorical_accuracy: 0.8469 - loss: 0.3740
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - categorical_accuracy: 0.9063 - loss: 0.2456
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - categorical_accuracy: 0.9703 - loss: 0.1078
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - categorical_accuracy: 0.9406 - loss: 0.1447
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - categorical_accuracy: 0.8553 - loss: 0.4857
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - categorical_accuracy: 0.9757 - loss: 0.1198
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - categorical_accuracy: 0.9579 - loss: 0.1627
Epoch 11/100
29/29 

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 30, 64)              │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 712,052 (2.72 MB)

 Trainable params: 237,350 (927.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 474,702 (1.81 MB)

In [21]:
res = model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step


In [22]:
actions[np.argmax(res[0])]

'help'

In [23]:
y_test

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
 

In [24]:
model.save('hands3.keras')

In [25]:
# model.load_weights('hands1.keras')

In [26]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [27]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [28]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[37,  0],
        [ 0, 11]],

       [[38,  0],
        [ 0, 10]],

       [[39,  0],
        [ 0,  9]],

       [[36,  0],
        [ 0, 12]],

       [[42,  0],
        [ 0,  6]]], dtype=int64)

In [29]:
accuracy_score(ytrue, yhat)

1.0

In [30]:
from scipy import stats

In [31]:
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (245,117,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [32]:
import cv2
import numpy as np
import mediapipe as mp
import time

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)

# Set Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

            # **Print confidence score**
            confidence = res[np.argmax(res)]
            print(f"Prediction: {actions[np.argmax(res)]}, Confidence: {confidence:.2f}")

            # 3. Visualization logic
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if confidence > threshold:
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualize probabilities
            image = prob_viz(res, actions, image, colors)

        # Display sentence on screen
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show output
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


C:\Users\andre\miniconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction: nothing, Confidence: 1.00
1/1 ━━━━━━━━